In [1]:
import os
import psycopg2
import psycopg2.extras
from sentence_transformers import SentenceTransformer
import pandas as pd

#  connection to PostgreSQL
connection = psycopg2.connect(
    dbname="your_db_name",
    user="your_username",
    password="your_password",
    host="localhost",
    port="5432"
)

#  embedding model using Hugging Face
model_name = "sentence-transformers/roberta-large-nli-stsb-mean-tokens"  # 1024-dimensional embeddings
embedding_model = SentenceTransformer(model_name)




read = pd.read_csv(r"C:\Users\atul.gupta\cleaned_new_pepsico.csv")

read

new_alis = read[['DB Field Name','Definition']].iloc[:3000]
new_alis['embedding'] = None
print(new_alis)



for index, row in new_alis.iterrows():
    field_dict = {}
    field_name = row['DB Field Name']
    definition = row['Definition']
    field_dict[field_name] = definition
    #print(field_dict)
    embeddings = embedding_model.encode([field_dict]).tolist()
    new_alis.at[index, 'embedding'] = embeddings[0]
    #print(embeddings)
new_alis


with connection.cursor() as cursor:
    for index,row in new_alis.iterrows():
        embedding_vector = row['embedding']
        #print(embedding_vector) 
        
        cursor.execute(
            """
            INSERT INTO test_3k_1024(db_field_name,definition,embedding)
            VALUES(%s,%s,%s)
            """,
            (row['DB Field Name'], row['Definition'], embedding_vector)
        )  
print("Done")
connection.commit()

c:\Users\atul.gupta\Anaconda3\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\atul.gupta\.cache\huggingface\hub\models--sentence-transformers--roberta-large-nli-stsb-mean-tokens. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\atul.gupta\Anaconda3\lib\site-packages\transformers\tokenization

                               DB Field Name  \
0        HID_Form_Status__Regen_Ag__Projects   
1     HID_Form_Status__Regen_Ag__Agriculture   
2     HID_Form_Status__Regen_Ag__Commodities   
3                 HID_Forms_Completion_Check   
4                                  Thank_you   
...                                      ...   
2995         MaterialMarketingTargetTypeCode   
2996                                   APLAL   
2997                                   PLNNR   
2998                                   STLAN   
2999                                   STLAL   

                                             Definition embedding  
0     Calculated value of overall form completion fo...      None  
1     Calculated value of overall form completion fo...      None  
2     Calculated value of overall form completion fo...      None  
3     Calculated value of overall form completion fo...      None  
4     Message is typically displayed on the last pag...      None  
...            

In [6]:
import numpy as np
import ast
from sentence_transformers import SentenceTransformer
import os
import psycopg2
import psycopg2.extras
from sentence_transformers import SentenceTransformer
import pandas as pd

#  connection to PostgreSQL
connection = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="Atul@123",
    host="localhost",
    port="5432"
)

#  embedding model using Hugging Face
model_name = "sentence-transformers/roberta-large-nli-stsb-mean-tokens"  # 1024-dimensional embeddings
embedding_model = SentenceTransformer(model_name)

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def search_similar_definitions(query, top_n=5):
    query_embedding = embedding_model.encode([query]).tolist()[0]

    with connection.cursor() as cursor:
        cursor.execute("SELECT db_field_name, definition, embedding FROM test_3k_1024")
        results = cursor.fetchall()
        

    similarities = []
    for row in results:
        db_field_name, definition, embedding_str = row
        
        
        embedding = ast.literal_eval(embedding_str)  
        
        similarity = cosine_similarity(query_embedding, embedding)
        similarities.append((db_field_name, definition, similarity))

    
    similarities.sort(key=lambda x: x[2], reverse=True)
    return similarities[:top_n]
#query = "Footer"
query = ["PageFooter", "AssetCondition","LastModifiedBy", "RegionalMarket", "Department","Nickname","LaunchDate", "CurrencySymbol", "EndOfPayrollPeriod","JobRquisitionId","DeathDate","ProbationEndDate","LastWorkingDate","ConfirmationDate","EmploymentTypeCode","NoticeOfTerminationDate","CorporateCreditCardNumber","TaiwanResignationDate","LocationCategoryCode","GeographicalZoneCode","LeadOwnerId","DivisionName"]

#query = ["Footer","asset_status","UpdatedById","MarketUnitName","JobFunctionName","PreferredName","GoLiveDate","CurrencyCode","PayrollEndDate","RequisitionId","DateOfDeath","ProbationPeriodEndDate","ResignationDate","ConfirmationDate","HireTypeCode","TerminationNotificationDate","CorporateCardNumber","ResignationTaiwanDate","LocationTypeCode","GeoZoneCode","PrimaryOwnerId","BusinessUnitName"]
for attribute in query:
    results = search_similar_definitions(attribute, top_n=5)
    print("Attribute_name:",attribute)
    for result in results:
        print(f"Field Name: {result[0]}, Definition: {result[1]}, Similarity: {result[2]}")
    print("\n")
    print("\n")
        


c:\Users\atul.gupta\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Attribute_name: PageFooter
Field Name: XTNOpenDayIndicator, Definition: The field contains the information of the opening day, Similarity: 0.6445300905243654
Field Name: SourceTableName, Definition: Represents system primary table name, Similarity: 0.551153608769502
Field Name: XTNPurchaseOrderStatusMessageIndicator, Definition: The field specifies the information of the message which is first post at the system, Similarity: 0.5379339972699014
Field Name: XTNAnnexingStartDate, Definition: The field contains the information about the start date, Similarity: 0.5291934108353035
Field Name: BIC_ZVISSTRD, Definition: The date when this visit begin, Similarity: 0.5198601038095482




Attribute_name: AssetCondition
Field Name: XTNAnnexingPackageKeyNumber, Definition: The number specifies the annexing which situated the key, Similarity: 0.438393750748891
Field Name: VBAP__ZAPCGKI, Definition: The number specifies the annexing which situated the key, Similarity: 0.438393750748891
Field Name: Pr

: 